In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os

In [125]:
#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
# users = pd.read_csv('../matrix_factorization/data/ml-100k/includes_team_users.csv').drop("Unnamed: 0",axis=1)

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('../data/ml-20m/ratings.csv')
smaller_ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')

#Reading items file:
# i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
# 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
# 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
# items = pd.read_csv('../matrix_factorization/data/ml-100k/u.item', sep='|', names=i_cols,
# encoding='latin-1')

items = pd.read_csv('../data/ml-20m/movies.csv')
smaller_items = pd.read_csv('../data/ml-latest-small/movies.csv')


In [126]:
ratings.head()
print(len(ratings))

20000263


In [117]:
smaller_ratings.head()
# print(len(smaller_ratings))

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [114]:
smaller_ratings.groupby("movieId").count().sort_values("userId").head()

,userId,rating,timestamp
movieId,,,
193609,1,1,1
4032,1,1,1
57526,1,1,1
57522,1,1,1
57502,1,1,1


In [6]:
items.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [40]:
smaller_items.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [102]:
print(len(smaller_items))
print(len(items))

9742
27278


In [66]:
users = pd.DataFrame({"userId":ratings.userId.unique()})
print(len(users))
users.head()

138493


,userId
0,1
1,2
2,3
3,4
4,5


In [69]:
users_smaller = pd.DataFrame({"userId":smaller_ratings.userId.unique()})
print(len(users_smaller))
users_smaller.head()

610


,userId
0,1
1,2
2,3
3,4
4,5


In [8]:
genres =[]
for i,movie in enumerate(items.values):
    movie_genre = movie[2].split("|")
    for genre in movie_genre:
        if genre not in genres:
            genres.append(genre)

In [9]:
genres

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'IMAX',
 'Documentary',
 'War',
 'Musical',
 'Western',
 'Film-Noir',
 '(no genres listed)']

In [29]:
for genre in genres:
    items[genre] = 0

In [30]:
items.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
def codeGenre(movie_id):
    genres = movie_id.genres.split("|")
    print(genres)
    for genre in genres:
        movie_id[genre] = 1
        

In [121]:
items.head()
# print(len(items))

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [42]:
genres = pd.get_dummies(items.genres.str.split('|',expand=True).stack()).sum(level=0)
genres.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
items_new = pd.concat([items,genres], axis=1)

In [54]:
items_new.drop(labels="genres",axis=1)

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,Heat (1995),0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6,7,Sabrina (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
7,8,Tom and Huck (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Sudden Death (1995),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [88]:
print(smaller_ratings.shape)

(100836, 4)


In [93]:
print(items_new.shape)

(27278, 23)


In [94]:
n_users = smaller_ratings.userId.unique().shape[0]
n_items = smaller_ratings.movieId.unique().shape[0]

In [95]:
n_users

610

In [96]:
n_items

9724

In [98]:
data_matrix = np.zeros((n_users, n_items))
for line in smaller_ratings.itertuples():
    print(line[1]-1, line[2]-1] = line[3])
#     data_matrix[line[1]-1, line[2]-1] = line[3]

IndexError: index 46969 is out of bounds for axis 1 with size 9724

In [119]:
smaller_ratings.iloc[46969]


userId       3.070000e+02
movieId      1.476000e+03
rating       2.500000e+00
timestamp    1.186163e+09
Name: 46969, dtype: float64